# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [126]:
# Dependencies and Setup
#!pip install gmaps  
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!jupyter nbextension enable --py --sys-prefix gmaps
#!jupyter nbextension enable --py gmaps

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json # remove after testing

# Import API key
from api_keys import g_key
#gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
# Import cities file as DataFrame
cities_pd = pd.read_csv("Output/city.csv")
 
cities_pd.head()


# #   "City": City,         #   *************Error on x_city************                  
#     "Cloudiness": Cloudiness,
#     "Country": Country,
#     "Date": Date,
#     "Humidity": Humidity,
#     "Lat": Lat,
#     "Lng": Lng,
#     "Max Temp": {'Max Temp'},
#     "Wind Speed": {'Wind Speed'}





# # Add columns for lat, lng, airport name, airport address, airport rating
# # Note that we used "" to specify initial entry.
# cities_pd["Airport Name"] = ""
# cities_pd["Airport Address"] = ""
# cities_pd["Airport Rating"] =
# cities_pd["Lat"] = ""
# cities_pd["Lng"] = ""
#  ""
# cities_pd.head()

,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,40,US,1586833936,58,20.89,-156.47,84.20,6.93
1,69,CK,1586833881,74,-21.21,-159.78,78.80,5.82
2,90,US,1586833886,53,42.60,-73.97,53.01,8.05
3,20,ZA,1586833583,87,-33.93,18.42,55.99,5.82
4,31,AU,1586833645,34,-33.65,115.33,73.99,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [105]:
# Store latitude and longitude in locations
locations = cities_pd[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = cities_pd["Humidity"].astype(float)

In [106]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [107]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
#xcities_pd = cities_pd.dropna(inplace = True)

ideal_weather_df = pd.DataFrame(cities_pd)     #, columns=(['Max Temp','Wind Speed','Cloudiness']))
ideal_weather_df.head()

max_temp = (ideal_weather_df['Max Temp'] < 80) & (ideal_weather_df['Max Temp'] > 70) 
wind_speed = ideal_weather_df['Wind Speed'] < 10
zero_cloudiness = ideal_weather_df['Cloudiness'] == 0


ideal_weather_df[max_temp & wind_speed & zero_cloudiness]

,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
74,0,AU,1586834220,38,-35.35,150.47,75.20,8.05
97,0,BR,1586834224,73,-21.48,-47.55,71.60,3.36
127,0,AU,1586834228,30,-33.38,148.02,77.00,1.61
236,0,IN,1586834244,33,12.46,75.96,79.93,3.94
274,0,BR,1586834249,81,-15.82,-50.61,72.37,2.95
308,0,AU,1586834254,38,-34.58,150.86,78.01,8.05
372,0,VE,1586834263,89,9.06,-62.05,71.91,3.91
393,0,AU,1586834145,47,-28.65,153.62,77.00,6.93
530,0,AU,1586833923,50,-32.72,152.15,79.00,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [120]:
hotel_df = ideal_weather_df.iloc[0:1] # **********get first row of dataframe for testing
hotel_df.head()

,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,40,US,1586833936,58,20.89,-156.47,84.2,6.93


In [128]:
params = {
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key,
}



# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():  #iterrows is a object that returns row

    # get lat and long type from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # combine lat & long
    params["location"] = f"{lat},{lng}"  # reference Airport_Ratings activity


   # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_data = requests.get(base_url, params=params)

    hotel_data = response.json()
    
    print(json.dumps(hotel_data, indent=4, sort_keys=True))
    


#  cities_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
#     cities_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# # Visualize to confirm lat lng appear
# cities_pd.head()




{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAI_Gu6oNbLvgMlVfB7LFF0fh39dQkMDruAXT6GGhWOJMwb_BkZN7e4Y2EPWndJQjN3IGc75mfifQKfzN6trNLfdLEXKOWXmhFm250dTmAi5OHWh3fOUMOBWiPZBvQhxsu4u54Ec7f4fgIkEbOpwH-hq3mtmN0pBIHVw2rIaPMtEf2WPAYy3F1yQTPqJLK2lvVrRDf14gZIFWYHbyD_ufyTdTI0SOVr0lZYdSs2GnPtfT9L6HPBEEi0uXhs2tJjGdaNBiKILGcJhofWPkPVmMDxzz0j9HUJ27EyM-qoZjqmQT6f58zKouiZFRF2UI1vicGyA8JBAYJcCuaVE3ew4FWZgMx2koYXKLcTwimLdm_v_MWE2tzDY9RJQGlButP8K5FhIQbwN34fB32qpF1yxCEyV9nhoUNnaLKwDaPBYWofyZA7TpD9nl7UA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 43.6042232,
                    "lng": -116.2137018
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.6056676302915,
                        "lng": -116.2123781697085
                    },
                    "southwest": {
                        "lat": 43.60296966970851,
                        "lng": -116.215076

In [102]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display Map